In [1]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50

In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from numba import jit
import time
import gc


In [3]:
@jit # 성능 향상
def eval_gini(y_true, y_prob):
  y_true = np.asarray(y_true) # 얇은 copy()
  y_true = y_true[np.argsort(y_prob)]
  ntrue = 0
  gini = 0
  delta = 0
  n = len(y_true)
  for i in range(n-1, -1, -1):
    y_i = y_true[i]
    ntrue += y_i
    gini += y_i * delta
    delta += 1 - y_i
  gini = 1 - 2 * gini / (ntrue * ( n - ntrue))
  return gini # 불순도 측정 지표, 높을 수록 분산도 높음


In [ ]:
def gini_xgb(preds, dtrain):
  labels = dtrain.get_label()
  gini_score = -eval_gini(labels, preds)
  return [('gini', gini_score)]

# 가우시안 정규 분포
def add_noise(series, noise_level):
  return series * (1 + noise_level + np.random.randn(len(series)))

def target_encode(trn_series=None, val_series=None, tst_series=None, \
  target=None, min_samples_leaf=1, smoothing=1, noise_level=0):

  assert len(trn_series) == len(target) # 예외 처리 구문
  assert trn_series.name == tst_series.name
  temp = pd.concat([trn_series, target], axis=1) # colunm
  averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
  smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
  prior = target.mean()

  averages[target.name] = prior * (1-smoothing) + averages["mean"] * smoothing
  averages.drop(["mean", "count"], axis=1, inplace=True)
  ft_trn_series = pd.merge(
    trn_series.to_frame(trn_series.name),
    averages.reset_index().rename(columns={'index': target.name, target.name:'average'}),
    on=trn_series.name,
    how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
  

In [5]:
train_df = pd.read_csv('./data/train.csv', na_values="-1", nrows=10000)
test_df = pd.read_csv('./data/test.csv', na_values="-1", nrows=10000)

In [9]:
print(train_df.iloc[0:200])

      id  target  ps_ind_01  ps_ind_02_cat  ps_ind_03  ps_ind_04_cat  \
0      7       0          2            2.0          5            1.0   
1      9       0          1            1.0          7            0.0   
2     13       0          5            4.0          9            1.0   
3     16       0          0            1.0          2            0.0   
4     17       0          0            2.0          0            1.0   
..   ...     ...        ...            ...        ...            ...   
195  493       0          1            1.0          7            0.0   
196  495       0          3            2.0          4            0.0   
197  496       0          2            1.0          6            0.0   
198  498       0          3            1.0          3            1.0   
199  502       0          1            1.0          5            1.0   

     ps_ind_05_cat  ps_ind_06_bin  ps_ind_07_bin  ps_ind_08_bin  ...  \
0              0.0              0              1              0

In [10]:
# from olivier
train_features = [
    "ps_car_13",  #            : 1571.65 / shadow  609.23
	"ps_reg_03",  #            : 1408.42 / shadow  511.15
	"ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
	"ps_ind_03",  #            : 1219.47 / shadow  230.55
	"ps_ind_15",  #            :  922.18 / shadow  242.00
	"ps_reg_02",  #            :  920.65 / shadow  267.50
	"ps_car_14",  #            :  798.48 / shadow  549.58
	"ps_car_12",  #            :  731.93 / shadow  293.62
	"ps_car_01_cat",  #        :  698.07 / shadow  178.72
	"ps_car_07_cat",  #        :  694.53 / shadow   36.35
	"ps_ind_17_bin",  #        :  620.77 / shadow   23.15
	"ps_car_03_cat",  #        :  611.73 / shadow   50.67
	"ps_reg_01",  #            :  598.60 / shadow  178.57
	"ps_car_15",  #            :  593.35 / shadow  226.43
	"ps_ind_01",  #            :  547.32 / shadow  154.58
	"ps_ind_16_bin",  #        :  475.37 / shadow   34.17
	"ps_ind_07_bin",  #        :  435.28 / shadow   28.92
	"ps_car_06_cat",  #        :  398.02 / shadow  212.43
	"ps_car_04_cat",  #        :  376.87 / shadow   76.98
	"ps_ind_06_bin",  #        :  370.97 / shadow   36.13
	"ps_car_09_cat",  #        :  214.12 / shadow   81.38
	"ps_car_02_cat",  #        :  203.03 / shadow   26.67
	"ps_ind_02_cat",  #        :  189.47 / shadow   65.68
	"ps_car_11",  #            :  173.28 / shadow   76.45
	"ps_car_05_cat",  #        :  172.75 / shadow   62.92
	"ps_calc_09",  #           :  169.13 / shadow  129.72
	"ps_calc_05",  #           :  148.83 / shadow  120.68
	"ps_ind_08_bin",  #        :  140.73 / shadow   27.63
	"ps_car_08_cat",  #        :  120.87 / shadow   28.82
	"ps_ind_09_bin",  #        :  113.92 / shadow   27.05
	"ps_ind_04_cat",  #        :  107.27 / shadow   37.43
	"ps_ind_18_bin",  #        :   77.42 / shadow   25.97
	"ps_ind_12_bin",  #        :   39.67 / shadow   15.52
	"ps_ind_14",  #            :   37.37 / shadow   16.65
]
# add combinations
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),  
    ('ps_reg_01', 'ps_car_04_cat'),
]